In [1]:
# from sklearn.datasets import fetch_20newsgroups
import pandas as pd

import os
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from pandarallel import pandarallel
import multiprocessing
multiprocessing.cpu_count()
pandarallel.initialize(progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
neg = pd.read_csv(r'../datasets/ffmpeg.csv')
pos = pd.read_csv(r'../datasets/qemu.csv')

In [3]:
df = pd.concat([neg[['commit_msg','patch','vulnerability']],pos[['commit_msg','patch','vulnerability']]],axis=0)
df.fillna('', inplace=True)
# 1是100%的意思
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
id2label={0:'Negative',1:'Positive'}
df.rename(columns={'vulnerability':'labels','patch':'diffs','commit_msg':'msgs'},inplace=True)
df = df.replace({"labels": id2label})
df 

,msgs,diffs,labels
0,jpeg2000dec: dont set ncomponents to an inval...,diff --git a/libavcodec/jpeg2000dec.c b/libavc...,Negative
1,block: convert ThrottleGroup to object with Q...,diff --git a/block/throttle-groups.c b/block/t...,Negative
2,MIPS: fix yield handling The parameter for yi...,diff --git a/target-mips/op_helper.c b/target-...,Negative
3,jvdec: avoid unsigned overflow in comparison ...,diff --git a/libavformat/jvdec.c b/libavformat...,Positive
4,qemu-img: print short help on getopt failure ...,diff --git a/qemu-img.c b/qemu-img.c\nindex c7...,Positive
...,...,...,...
25868,migration: don't close a file descriptor whil...,diff --git a/migration/migration.c b/migration...,Positive
25869,"Simplify left_xy content for the loop filter,...",diff --git a/libavcodec/h264.h b/libavcodec/h2...,Negative
25870,"Fix headers so that ""make checkheaders"" passe...",diff --git a/libavcodec/amr.h b/libavcodec/amr...,Negative
25871,intel-iommu: Check IOAPIC's Trigger Mode agai...,diff --git a/hw/i386/intel_iommu.c b/hw/i386/i...,Negative


In [4]:
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [ ]:




def process_row(row):
    from openai import OpenAI
    import tiktoken
    client = OpenAI(
        api_key="sk-bIHAQMOPc0mMMFKgkvZE9sMyggQvA6zi1SR6oq7uSyZRQpc5",  # 或者 os.environ["OPENAI_API_KEY"]
        base_url="https://sg.uiuiapi.com/v1",  # 注意最好加 /v1,
        # model="gpt-4"
    )
    # Helper method to prompt OpenAI LLM and get response.
    def get_response(user_content):
        response = client.chat.completions.create(
        model="gpt-4",  # 根据代理方文档修改
        messages=[
            {"role": "user", "content": user_content}
        ]
        )
        print(response.choices[0].message.content)
        
        return response.choices[0].message.content

    # text = "\'How can I change my pin?\'"
    # examples = get_examples(examples_pool)
    # df = pd.read_csv('dataset.csv')
    # Helper to format the k-shot prompt with:
    # - prefix
    # - 1 example from each class
    # - target text for classification
    def num_tokens_from_messages(messages, model="gpt-4-0301"):
        """Returns the number of tokens used by a list of messages."""
        try:
            encoding = tiktoken.encoding_for_model(model)
        except KeyError:
            encoding = tiktoken.get_encoding("cl100k_base")
        if model == "gpt-4-0301":  # note: future models may deviate from this
            num_tokens = 0
            for message in messages:
                num_tokens += (
                    4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
                )
                for key, value in message.items():
                    num_tokens += len(encoding.encode(value))
                    if key == "name":  # if there's a name, the role is omitted
                        num_tokens += -1  # role is always required and always 1 token
            num_tokens += 2  # every reply is primed with <im_start>assistant
            return num_tokens
        else:
            raise NotImplementedError(
                f"""num_tokens_from_messages() is not presently implemented for model {model}.
    See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
            )

    def generate_promot(row):
        base_promot = """
        Please act as a security patch identificator,
        categorize patch into two categories: Negative, Positive.
        The Negative category corresponds to patches which includes insecure code changes.
        The Positive category corresponds to patches which includes secure code changes.
        I will provide you with the commit message and code diff for a commit patch,
        and you need to give me the category label for this commit patch.
        Please avoid any explanations and only provide the category label.
        """

        commit_message_prompt = f"commit message: {row['msgs']}\n"
        commit_codediff_prompt = f"code diff: {row['diffs']}\n"

        prompt = base_promot + commit_message_prompt + commit_codediff_prompt

        messages = [{"role": "user", "content": prompt}]

        l, r = 0, len(commit_codediff_prompt) - 1
        while l <= r:
            mid = (l + r) // 2
            prompt = base_promot + commit_message_prompt + commit_codediff_prompt[:mid]
            messages = [{"role": "user", "content": prompt}]

            ok = num_tokens_from_messages(messages) < 4096
            if ok:
                l = mid + 1
            else:
                r = mid - 1
        prompt = base_promot + commit_message_prompt + commit_codediff_prompt[:r]

        return prompt
    user_content = generate_promot(row)


    return get_response(user_content)

val_df['result'] = val_df.parallel_apply(process_row, axis=1)


In [6]:
val_df['result']

15786    Positive
18511    Positive
16506    Positive
6547     Positive
15040    Positive
           ...   
1371     Negative
1604     Positive
22465    Positive
19917    Negative
25080    Positive
Name: result, Length: 3881, dtype: object

In [7]:
val_df['labels']

15786    Negative
18511    Negative
16506    Positive
6547     Negative
15040    Negative
           ...   
1371     Negative
1604     Negative
22465    Negative
19917    Positive
25080    Negative
Name: labels, Length: 3881, dtype: object

In [8]:
val_df.to_csv('SPIDB_result.csv',encoding='utf_8_sig')

In [ ]:
# 仅保留 result 为 'positive' 或 'negative' 的行
val_df = val_df[val_df['result'].isin(['positive', 'negative'])].copy()

# 打印对齐检查（行数应一致）
print(f"After filtering, len(result)={len(val_df['result'])}, len(labels)={len(val_df['labels'])}")

# 比较结果是否一致
comparison = val_df['result'] == val_df['labels']

# 查看不一致的情况
mismatch_df = val_df[~comparison]

print(f"共有 {len(mismatch_df)} 行不匹配。")
display(mismatch_df.head())

In [28]:
val_df = pd.read_csv('SPIDB_result.csv')
val_df

,Unnamed: 0,msgs,diffs,labels,result
0,15786,Fix lossless jpeg encoder to comply to spec a...,diff --git a/libavcodec/ljpegenc.c b/libavcode...,Negative,Positive
1,18511,rtl8139: add format attribute to DPRINTF gcc ...,diff --git a/hw/rtl8139.c b/hw/rtl8139.c\ninde...,Negative,Positive
2,16506,scsi-generic: fix sign extension of READ CAPA...,diff --git a/hw/scsi/scsi-generic.c b/hw/scsi/...,Positive,Positive
3,6547,tests: No longer dependent on CONFIG_UUID cry...,diff --git a/tests/test-crypto-block.c b/tests...,Negative,Positive
4,15040,main-loop: drop rfds/wfds/xfds for good Now t...,diff --git a/main-loop.c b/main-loop.c\nindex ...,Negative,Positive
...,...,...,...,...,...
3876,1371,Let the venomous poison spread to more identi...,diff --git a/hw/poison.h b/hw/poison.h\nindex ...,Negative,Negative
3877,1604,hw/acpi: piix4_pm_init(): take fw_cfg object ...,diff --git a/hw/acpi/piix4.c b/hw/acpi/piix4.c...,Negative,Positive
3878,22465,g723.1: replace loop of normalize_bits( by av...,diff --git a/libavcodec/g723_1.c b/libavcodec/...,Negative,Positive
3879,19917,tests: fix migration-test leak Direct leak of...,diff --git a/tests/migration-test.c b/tests/mi...,Positive,Negative


In [31]:
val_df[val_df['result']=='Negative.']

,Unnamed: 0,msgs,diffs,labels,result
2366,15952,lavu: disable wrong value check in get_versio...,diff --git a/libavutil/utils.c b/libavutil/uti...,Negative,Negative.


In [32]:
val_df['result'] = val_df['result'].replace({'Negative.': 'Negative', 'Positive': 'Positive','Negative':'Negative'})

In [33]:
val_df['result']

0       Positive
1       Positive
2       Positive
3       Positive
4       Positive
          ...   
3876    Negative
3877    Positive
3878    Positive
3879    Negative
3880    Positive
Name: result, Length: 3881, dtype: object

In [34]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(val_df['labels'],val_df['result'],digits=4))

              precision    recall  f1-score   support

    Negative     0.5172    0.1656    0.2508      2265
    Positive     0.4011    0.7834    0.5306      1616

    accuracy                         0.4228      3881
   macro avg     0.4592    0.4745    0.3907      3881
weighted avg     0.4689    0.4228    0.3673      3881



NameError: name 'test_labels' is not defined

In [ ]:
1

1